# Analysis of Google app for work Log Tutorial (0)

# Step 1: Connection and simple Query (TermQuery and RangeQuery)

In [ ]:
import pyes
import elasticsearch

es_address='114.32.24.166:9200'
conn = pyes.es.ES(es_address)
bq = pyes.query.BoolQuery() 
#q = pyes.query.MatchAllQuery()

tq = pyes.query.TermQuery(field="actor_email", value="ericmao@cyber00rn.org")

ESR = pyes.ESRange(field="id_time", from_value="2016-03-01", to_value="2016-04-30", 
                   include_lower=True ,include_upper=False)
rq = pyes.query.RangeQuery(qrange=ESR)

bq.add_must(tq)
bq.add_must(rq)

result = conn.search(query = bq , indices = 'google' , doc_types = 'drive') 
len(result)

# Step 2: How to use Set to arrange the distinct value

In [5]:
country_set=set()
for i in range(0,100):
    if 'geoip_countryCode' in result[i]:
        country_set.add(result[i]['geoip_countryCode'])

len(country_set)
country_set

ip_set=set()
for i in range(0,100):
    if 'ipAddress' in result[i]:
        ip_set.add(result[i]['ipAddress']+","+result[i]['geoip_countryCode'])

len(ip_set)
ip_set

geo_set=set()
for i in range(0,len(result)):
    if 'ipAddress' in result[i]:
        geo_set.add(str(result[i]['geoip_latitude'])+","+str(result[i]['geoip_longitude']))

len(geo_set)
geo_set

{u'101.12.65.229,TW',
 u'140.92.146.23,TW',
 u'140.92.253.17,TW',
 u'180.206.2.56,TW',
 u'180.206.2.57,TW',
 u'211.73.74.33,TW',
 u'211.73.74.35,TW',
 u'49.214.49.175,TW',
 u'49.216.65.186,TW',
 u'49.217.241.188,TW'}

# Step 3: View the detail of each record

In [64]:
result[0]

{'actor_email': 'ericmao@cyber00rn.org',
 'actor_profileId': '108189844280408676879',
 'event_destination_folder_id': ['0BwHnaad0ivbsUkhWS0ZZLWh6MEk'],
 'event_destination_folder_title': [u'\u5206\u4eab\u8b1b\u7fa9'],
 'event_dlp_info': u'',
 'event_doc_id': '0BwHnaad0ivbsOUVCT0dwRWcyc0E',
 'event_doc_title': 'Malware Analysis Fundamentals (1).pptx',
 'event_doc_type': 'unknown',
 'event_name': 'add_to_folder',
 'event_owner': 'ericmao@cyber00rn.org',
 'event_primary_event': False,
 'event_type': 'access',
 'geoip_countryCode': 'TW',
 'geoip_countryName': 'Taiwan',
 'geoip_latitude': 23.5,
 'geoip_longitude': 121,
 'id_applicationName': 'drive',
 'id_customerId': 'C02u7zbcq',
 'id_time': '2016-03-20T12:29:13.987Z',
 'id_uniqueQualifier': 8245649586173182973,
 'ipAddress': '180.206.8.146',
 'kind': 'admin#reports#activity'}

# Step 4: Single layer data aggregation

In [2]:
import json

conn = pyes.es.ES('localhost:9200')
q = pyes.query.MatchAllQuery()
# aggregation
tagg = pyes.aggs.TermsAgg('actor_email', field = 'actor_email') 

qsearch = pyes.query.Search(q) 

qsearch.agg.add(tagg) 

rs = conn.search(query = qsearch,index = 'google',type = "drive") 
print (json.dumps(rs.aggs, indent = 2)) 

NameError: name 'pyes' is not defined

# Step 5: Multi layer data aggregation

In [70]:
import pyes
import json
conn = pyes.es.ES('localhost:9200')
q = pyes.MatchAllQuery()
tagg = pyes.aggs.TermsAgg('user', field = 'actor_email', sub_aggs = []) 
tagg1 = pyes.aggs.TermsAgg('country', field = 'geoip_countryName')  
tagg.sub_aggs.append(tagg1) 
# 將tagg1加到tagg.sub_aggs裡面。
qsearch = pyes.query.Search(q) 
# This "Search" is under pyes.query, http://pydoc.net/Python/pyes/0.99.5/pyes.query
qsearch.agg.add(tagg)

rs = conn.search(query = qsearch , indices = 'google' ,type = "drive" )
print (json.dumps(rs.aggs, indent = 2))

{
  "user": {
    "buckets": [
      {
        "country": {
          "buckets": [
            {
              "key": "Taiwan", 
              "doc_count": 262908
            }, 
            {
              "key": "Japan", 
              "doc_count": 33
            }
          ], 
          "doc_count_error_upper_bound": 0, 
          "sum_other_doc_count": 0
        }, 
        "key": "maokao@cyber00rn.org", 
        "doc_count": 263065
      }, 
      {
        "country": {
          "buckets": [
            {
              "key": "Taiwan", 
              "doc_count": 169045
            }
          ], 
          "doc_count_error_upper_bound": 0, 
          "sum_other_doc_count": 0
        }, 
        "key": "ted@cyber00rn.org", 
        "doc_count": 169170
      }, 
      {
        "country": {
          "buckets": [
            {
              "key": "Taiwan", 
              "doc_count": 58347
            }
          ], 
          "doc_count_error_upper_bound": 0, 
          "sum_ot

In [71]:
qsearch

{'query': {'match_all': {}}, 'aggs': {'user': {'terms': {'field': 'actor_email', 'size': 100}, 'aggs': {'country': {'terms': {'field': 'geoip_countryName', 'size': 100}}}}}}